In [1]:
#1) Detect when vehicle is being used  
#2) Match driver's phone and vehicle
#3) Track driving behavior
#4) Track if texting is enabled
#5) Run scoring model, on risky and distracted events
#6) Make data available for Scoring API

In [2]:
import sys
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
from geopy import distance
from dateutil.parser import parse
from datetime import datetime
import math
import boto3
#!{sys.executable} -m pip install geopy

In [22]:
#Helper Functions
def parse_time(s):
    try:
        ret = parse(s)
    except ValueError:
        ret = datetime.utcfromtimestamp(s)
    return ret

#print(parse_time(data['lastTimestamp'][0]))

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c
    return d

#distance([-88.3185833333333, 42.228175],[-88.3185833333333, 48.228175])

In [3]:
cd Downloads

/Users/wandawang/Downloads


In [4]:
##Call the API --- #TelemetryByVinController #Header Parameters #x-id-token

def api_call(fullpath, parameters):
    session = requests.Session() #performance increase if make several requests to same host 

    response = session.get(url=fullpath, params=parameters)
    #response = requests.get("https://jsonplaceholder.typicode.com/todos")
    todos = json.loads(response.text) #dict
    #OR
    data = response.json() #dict
    #print(type(data)) 
    #print(json_normalize(data)) #dataframe 
    print(type(json_normalize(data)))
    #print(data.keys())
    if response.status_code != 200:
        print("Cannot connect to API source.")

In [18]:
#Voyo example
#https://prod001.voyomotive.com/api/all.php?serial=[SERIAL]&key=[API_KEY]&start=[START_TIME]&end=[END_TIME]
fullpath = "https://prod001.voyomotive.com/api/all.php"
parameters = {'serial': '6468','key': 'X8KFUACqQGix1DTbNRov','start': 1569801600,'end': 1569888000}
api_call(fullpath, parameters)

<class 'pandas.core.frame.DataFrame'>


In [6]:
##Assuming the API returns a JSON, parse the JSON object into a Python dict using json.loads function
    #Grab vin, lastTimestamp, location-coordinates, telemetry-speed-value
    
with open('telemetry_api_response.txt') as f: 
    json_data = json.load(f)
json_data

##Loop through the dict to extract information.

{'vin': 'VINVINVINVINVIN01',
 'correlationId': 'correlation-id',
 'lastTimestamp': '2018-06-02T22:04:18Z',
 'location': {'type': 'Point', 'coordinates': [-88.3185833333333, 42.228175]},
 'vehicleName': 'CAMRY',
 'telemetry': {'fuelLevel': {'value': 26.46},
  'odometer': {'value': 5176, 'unit': 'mi'},
  'sunRoof': {'value': 2},
  'spareTirePressure': {'value': 127, 'unit': 'psi'},
  'flTirePressure': {'value': 36, 'unit': 'psi'},
  'driverWindow': {'value': 2},
  'rrWindow': {'value': 2},
  'speed': {'value': 0, 'unit': 'km/h'},
  'frTirePressure': {'value': 37, 'unit': 'psi'},
  'rlTirePressure': {'value': 37, 'unit': 'psi'},
  'passengerWindow': {'value': 2},
  'rrTirePressure': {'value': 37, 'unit': 'psi'},
  'rlWindow': {'value': 2}}}

In [19]:
data = json_normalize(json_data)

In [15]:
#data.columns

In [20]:
df = data[['vin','vehicleName','lastTimestamp','location.coordinates','telemetry.odometer.value','telemetry.speed.value']]
df

,vin,vehicleName,lastTimestamp,location.coordinates,telemetry.odometer.value,telemetry.speed.value
0,VINVINVINVINVIN01,CAMRY,2018-06-02T22:04:18Z,"[-88.3185833333333, 42.228175]",5176,0


In [21]:
df_repeated = pd.concat([df]*3, ignore_index=True)
df_repeated

,vin,vehicleName,lastTimestamp,location.coordinates,telemetry.odometer.value,telemetry.speed.value
0,VINVINVINVINVIN01,CAMRY,2018-06-02T22:04:18Z,"[-88.3185833333333, 42.228175]",5176,0
1,VINVINVINVINVIN01,CAMRY,2018-06-02T22:04:18Z,"[-88.3185833333333, 42.228175]",5176,0
2,VINVINVINVINVIN01,CAMRY,2018-06-02T22:04:18Z,"[-88.3185833333333, 42.228175]",5176,0


In [24]:
#clean timestamps

data = df_repeated[['vin','vehicleName','lastTimestamp','location.coordinates','telemetry.odometer.value','telemetry.speed.value']]
data['lastTimestamp'] = data['lastTimestamp'].apply(lambda x: parse_time(x))
data

,vin,vehicleName,lastTimestamp,location.coordinates,telemetry.odometer.value,telemetry.speed.value
0,VINVINVINVINVIN01,CAMRY,2018-06-02 22:04:18+00:00,"[-88.3185833333333, 42.228175]",5176,0
1,VINVINVINVINVIN01,CAMRY,2018-06-02 22:04:18+00:00,"[-88.3185833333333, 42.228175]",5176,0
2,VINVINVINVINVIN01,CAMRY,2018-06-02 22:04:18+00:00,"[-88.3185833333333, 42.228175]",5176,0


In [ ]:
# upload table into s3 bucket and folder location


#aws s3 parameters
s3 = boto3.resource('s3')
client = boto3.client('s3')
bucket = s3.Bucket('tri-avt') #bucket name
session = boto3.Session()
s3_client = session.client('s3')


#read from s3

#Loading dataframes per file type
def load_files(folder, filename): #folder = 'sample', filename = 'synced_can.csv'
    file_list = []
    for key in bucket.objects.filter(Prefix=folder):
        k = key.key
        if k.endswith(filename):
            file_list.append(k)
            
    list_data = []
    for key in file_list: #[0:100]: #looking at first 10 files only
        file_body = s3_client.get_object(Bucket='tri-avt', Key=key).get('Body')
        n = (re.findall("\d+", key))
        filename = (n[0]+ '_'+ n[1] + '_' + n[2])
        df = pd.read_csv(io.BytesIO(file_body.read()))
        df['fname'] = filename
        list_data.append(df)

    frame_data = pd.concat(list_data, axis = 0, ignore_index = True)
    return(frame_data)

#speed threshold
#gps_coord - geofencing, match with phone
#odo - distance traveled


##1) Detect when vehicle is being used  
#When Speed > 0 mph

#Tag Driver -- Assign Driver_ID
#Tag Trips -- identify time gaps of > 10 mins , Assign Trip_ID
#Tag Phone -- identify Phone_ID, match gps_coord with phone gps_coord. 


In [17]:
#center_point = [{'lat': -7.7940023, 'lng': 110.3656535}]
#test_point = [{'lat': -7.79457, 'lng': 110.36563}]

center_point = [{'lat': 33.85750, 'lng': -118.28885}] #k1 speed
test_point = [{'lat': 33.85717, 'lng': -118.29022}] #green olive

radius = .05 # in kilometer

center_point_tuple = tuple(center_point[0].values()) # (-7.7940023, 110.3656535)
test_point_tuple = tuple(test_point[0].values()) # (-7.79457, 110.36563)

dis = distance.distance(center_point_tuple, test_point_tuple).km
print("Distance: {}".format(dis)) # Distance: 0.0628380925748918

if dis <= radius:
    print("{} point is inside the {} km radius from {} coordinate".format(test_point_tuple, radius, center_point_tuple))
else:
    print("{} point is outside the {} km radius from {} coordinate".format(test_point_tuple, radius, center_point_tuple))

Distance: 0.13195669782039013
(33.85717, -118.29022) point is outside the 0.05 km radius from (33.8575, -118.28885) coordinate


In [16]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

center_point = [{'lat': 33.85750, 'lng': -118.28885}] #k1 speed
test_point = [{'lat': 33.85717, 'lng': -118.29022}] #green olive

lat1 = center_point[0]['lat']
lon1 = center_point[0]['lng']
lat2 = test_point[0]['lat']
lon2 = test_point[0]['lng']

radius = 0.01 # in kilometer #0.01 km = 10 meters

a = haversine(lon1, lat1, lon2, lat2)

print('Distance (km) : ', a)
if a <= radius:
    print('Inside the area')
else:
    print('Outside the area')

Distance (km) :  0.13171921744122636
Outside the area


In [ ]:
#Distracted Driving
#When Speed > 30 mph
#Geofencing logic
#Locked Miles 

In [ ]:
#ToDos

#Obtain data stream of phone GPS coordinates 
#

In [ ]:
#2) Match driver's phone and vehicle

#Requirements -- need coordinates of phone provided.
# or vehicle coordinates provided to phone or notebook.
# function to match, every 5 seconds? 

#Check if Phone GPS Coordiantes is within 10m radius of Veicle gps_coord 
data['gps_coord'][0][0]

In [ ]:
#3) Track driving behavior
#4) Track if texting is enabled
#5) Run scoring model, on risky and distracted events

In [13]:
#df = pd.DataFrame(columns=['lon', 'lat', 'time', 'speed'])

In [14]:
#AutoML -- what am I trying to predict?